# Data science projects for business - Final Assignment

* Filesi Gianluca 102299
* O' Donovan Luke 102312

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import seaborn as sns
from scipy.stats import skew, kurtosis
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import pickle

In [ ]:
df_test = pd.read_csv('Notebook/Filesi_ODonovan.ipynb')
df_train = pd.read_csv('Notebook/train.csv')

In [ ]:
df_train.head()

In [ ]:
numerical_columns = df_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
non_numerical_columns = df_train.select_dtypes(exclude=['int64', 'float64']).columns.tolist()

print("Numerical Columns:", numerical_columns)
print("Non-Numerical Columns:", non_numerical_columns)


## Data visualization

In [ ]:
# Visualize distributions
df_train.hist(figsize=(12,8))
plt.show()

In [ ]:
numerical_cols = df_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_cols = numerical_columns[1:]

In [ ]:
# Visualizing distributions of numerical features
fig, axes = plt.subplots(len(numerical_cols)//3, 3, figsize=(15, 5))
axes = axes.flatten()
for i, col in enumerate(numerical_cols):
    sns.histplot(df_train[col].dropna(), bins=30, kde=True, ax=axes[i-1])
    axes[i-1].set_title(f"Distribution of {col}")
plt.tight_layout()
plt.show()

In [ ]:
# Checking skewness and kurtosis for numerical columns
print("Skewness & Kurtosis for Numerical Features:")
for col in numerical_cols:
    print(f"{col}: Skewness = {skew(df_train[col].dropna()):.2f}, Kurtosis = {kurtosis(df_train[col].dropna()):.2f}")

In [ ]:
# Boxplots for detecting outliers
fig, axes = plt.subplots(len(numerical_cols)//3, 3, figsize=(15, 5))
axes = axes.flatten()
for i, col in enumerate(numerical_cols):
    sns.boxplot(y=df_train[col], ax=axes[i-1])
    axes[i-1].set_title(f"Boxplot of {col}")
plt.tight_layout()
plt.show()

## Data preprocessing

- Engine volume: convert to numbers + dummy variable for turbo ones

- Levy: convert to number

- Mileage: remove km to convert them in figures

In [ ]:
# EDA: Check for missing values
print(df_train.isnull().sum())

In [ ]:
def converting(df):
    # Convert Levy to numeric (handling missing values)
    df['Levy'] = pd.to_numeric(df['Levy'], errors='coerce').fillna(0)

    # Convert Mileage to numeric (removing non-numeric characters)
    df['Mileage'] = df['Mileage'].str.replace(r'\D+', '', regex=True).astype(float)

    # Create a dummy column for "turbo" (1 if "turbo" is present, else 0)
    df['turbo'] = df['Engine volume'].str.contains('turbo', case=False, na=False).astype(int)
    df['Engine volume'] = df['Engine volume'].str.extract(r'(\d+)').astype(int)
    return df

df_train = converting(df_train)
df_test = converting(df_test)

### Encoding

In [ ]:
OHE_cols = ['Category', 'Fuel type', 'Gear box type', 'Drive wheels', 'Doors', 'Color'] 
Binary_cols = ['Leather interior', 'Wheel']
Num_cols = [x for x in df_train.columns if x not in OHE_cols + Binary_cols]

In [ ]:
print(Num_cols)

In [ ]:
Num_cols.remove('Model')
Num_cols.remove('Manufacturer')
Num_cols.remove('turbo')
Num_cols.remove('ID')

In [ ]:
# Compute correlation matrix
correlation_matrix = df_train[Num_cols].corr()
np.fill_diagonal(correlation_matrix.values, np.nan)

# Visualize with heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
plt.title("Feature Correlation Matrix")
plt.show()


## Detecting Outliers

In [ ]:
# Visualize outliers
for col in Num_cols:
    plt.figure(figsize=(5, 4))  
    df_train[col].plot(kind='box', vert=True, title=f"Boxplot of {col}")  
    plt.ylabel(col)
    plt.show()

In [ ]:
features = ['Levy', 'Price', 'Engine volume', 'Mileage']

for feature in features:
    top_10 = df_train[feature].nlargest(10)  # Get top 10 largest values
    print(f"Top 10 Maximum {feature}:\n{top_10}\n")


In [ ]:
df_cleaned = df_train.copy()

In [ ]:
Q1 = df_train['Mileage'].quantile(0.10)  # First quartile (15th percentile)
Q3 = df_train['Mileage'].quantile(0.90)  # Third quartile (85th percentile)
IQR = Q3 - Q1  # Interquartile range

# Define bounds
lower_bound = Q1 - 1.8 * IQR
upper_bound = Q3 + 1.8 * IQR

# Remove outliers
df_cleaned = df_cleaned[df_cleaned['Mileage'] <= upper_bound]

### Remove top value of Levy , Price and engine volume 

In [ ]:
# Identify the max values for each feature
max_price = df_train['Price'].max()
max_engine_volume = df_train['Engine volume'].max()

# Remove rows where:
# - Levy is greater than 11,000
# - Price is the maximum value
# - Engine volume is the maximum value
df_cleaned = df_cleaned[
    (df_cleaned['Levy'] <= 11000) &  # Remove rows where Levy > 11,000
    (df_cleaned['Price'] != max_price) & 
    (df_cleaned['Engine volume'] != max_engine_volume)
]


In [ ]:
# Visualize outliers
for col in Num_cols:
    plt.figure(figsize=(5, 4))  
    df_cleaned[col].plot(kind='box', vert=True, title=f"Boxplot of {col}")  
    plt.ylabel(col)
    plt.show()

In [ ]:
df_train_OLD = df_train
df_train = df_cleaned

## Categorical to Dummies

In [ ]:
def enc(df):
    encoder = OneHotEncoder(sparse=False) 

    # Fit and transform the selected columns
    encoded_array = encoder.fit_transform(df[OHE_cols])

    # Convert to DataFrame and retain column names
    encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(OHE_cols))

    # Drop original categorical columns and concatenate the new encoded columns
    df = df.drop(OHE_cols, axis=1).reset_index(drop=True)
    df = pd.concat([df, encoded_df], axis=1)

    encoder = OneHotEncoder(sparse=False,drop='if_binary')

    # Fit and transform the selected columns
    encoded_array = encoder.fit_transform(df[Binary_cols])

    # Convert to DataFrame and retain column names
    encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(Binary_cols))

    # Drop original categorical columns and concatenate the new encoded columns
    df = df.drop(Binary_cols, axis=1).reset_index(drop=True)
    df = pd.concat([df, encoded_df], axis=1)

    return df

In [ ]:
def Manu(df,percentage):
    manufacturer_percentage = df['Manufacturer'].value_counts(normalize=True).mul(100).round(2)

    # First, sort the manufacturer percentages in descending order
    sorted_manufacturers = manufacturer_percentage.sort_values(ascending=False)

    # Compute cumulative sum of these percentages
    cumulative = sorted_manufacturers.cumsum()

    # Select manufacturers that together account for up to 90% of the data
    manufacturers_90 = sorted_manufacturers[cumulative <= percentage]

    manufacturers = pd.concat([manufacturers_90, pd.Series({'Other': 100-manufacturers_90.sum()})], axis=0)

    # Replace manufacturers not in the top 90% with 'Other'
    df['Manufacturer'] = df['Manufacturer'].apply(lambda x: x if x in manufacturers_90.index else 'Other')

    # Create dummy variables (1 for presence, 0 for absence)
    manufacturer_dummies = pd.get_dummies(df['Manufacturer'], prefix='Manufacturer').astype(int)

    # Concatenate the dummy variables with the original dataset
    df = pd.concat([df, manufacturer_dummies], axis=1)

    # Drop the original 'Manufacturer' column (optional)
    df.drop(columns=['Manufacturer'], inplace=True)

    return df

### Manufacteur % - Top 13 account for 90%

In [ ]:
df_train = enc(df_train)
df_train = Manu(df_train,90)

df_test = enc(df_test)
df_test = Manu(df_test,90)

## Converted top 90% of manumanufacturers to dummy variables 

In [ ]:
df_train.head()

In [ ]:
df_test.head()

## Drop model column as there is too many unique models to account for 

In [ ]:
df_train = df_train.drop(columns=['Model'])
df_train = df_train.drop(columns=['ID'])

df_test = df_test.drop(columns=['Model'])
df_test = df_test.drop(columns=['ID'])

In [ ]:
df_train.head()

# Baseline Model 

In [ ]:
df_draft = df_train[['Mileage', 'Price']]
df_draft = df_draft.sort_values(by='Mileage', ascending=True)

In [ ]:
plt.plot(df_draft['Mileage'], df_draft['Price'])
plt.xlabel('Mileage')
plt.ylabel('Price')
plt.show()

In [ ]:
# Define features (X) and target (y)
log_mil = np.log(df_train['Mileage']+1)
X = df_train.drop(columns=["Price"])
X["Mileage"] = log_mil
y = df_train["Price"]

# Split data into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test)  

# Train a Linear Regression model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print results
print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")


# Experiment with different ML models and compare 

In [ ]:
# Define features (X) and target (y)
log_mil = np.log(df_train['Mileage']+1)
X = df_train.drop(columns=["Price"])
X["Mileage"] = log_mil
y = df_train["Price"]

# Split data into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test)  

# Define models
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Decision Tree": DecisionTreeRegressor(random_state=42, max_depth=5),
    "Random Forest": RandomForestRegressor(n_estimators=50, random_state=42, max_depth=5),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=50, random_state=42, max_depth=5),
}

# Store results
results = []

# Train and evaluate each model
for name, model in models.items():
    # Train model
    model.fit(X_train_scaled, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    
    # Evaluate
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)  

    
    # Store results
    results.append({"Model": name, "MSE": mse, "R² Score": r2, "RMSE": rmse})

# Convert results to DataFrame and print
results_df = pd.DataFrame(results)
print(results_df)


In [ ]:
# Define features (X) and target (y)
log_mil = np.log(df_train['Mileage']+1)
X = df_train.drop(columns=["Price"])
X["Mileage"] = log_mil
y = df_train["Price"]

# Split data into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test)  

# Define parameter grid for Gradient Boosting
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.1, 0.2]
}

# Reduce CV folds if dataset is small
cv_splits = min(3, len(y_train))

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=GradientBoostingRegressor(random_state=42),
    param_grid=param_grid,
    scoring="neg_mean_squared_error",
    cv=cv_splits,  
    n_jobs=-1
)

# Perform GridSearchCV
grid_search.fit(X_train_scaled, y_train)

# Get best parameters and best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Evaluate the best model on test data
y_pred_best = best_model.predict(X_test_scaled)
best_mse = mean_squared_error(y_test, y_pred_best)
best_r2 = r2_score(y_test, y_pred_best)
best_rmse = best_mse ** 0.5

# Print results
print("Best Hyperparameters:", best_params)
print(f"Best Model MSE: {best_mse}")
print(f"Best Model RMSE: {best_rmse}")
print(f"Best Model R² Score: {best_r2}")


## Hyperparmater Tuning - Grid Seach 

In [ ]:
# Print Best Hyperparameters
print("Best Hyperparameters:", best_params)

# Evaluate Best Model on Test Data
y_pred_best = best_model.predict(X_test_scaled)
best_mse = mean_squared_error(y_test, y_pred_best)
best_r2 = r2_score(y_test, y_pred_best)
best_rmse = np.sqrt(best_mse)

# Print Performance Metrics
print(f"Best Model MSE: {best_mse}")
print(f"Best Model RMSE: {best_rmse}")
print(f"Best Model R² Score: {best_r2}")

# Get feature importances
feature_importance = best_model.feature_importances_
features = X.columns  

# Sort Feature Importances 
sorted_idx = np.argsort(feature_importance)[-20:]  

# Plot Top 20 Feature Importances
plt.figure(figsize=(10, 6))
plt.barh(range(20), feature_importance[sorted_idx], align="center")
plt.yticks(range(20), [features[i] for i in sorted_idx])
plt.xlabel("Feature Importance")
plt.ylabel("Features")
plt.title("Top 20 Feature Importance in Gradient Boosting Model")
plt.show()



## Model Saving 

In [ ]:
# Define the filename to save the model
model_filename = "best_gradient_boosting_model.pkl"

# Save the model using pickle
with open(model_filename, 'wb') as file:
    pickle.dump(best_model, file)

print(f"Model saved successfully as {model_filename}")


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1a77229b-6e4b-484a-822b-79f4c1fa6299' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>